The code is written in python 3.7.5, it doesn't require any 3rd party libraries or numpy


Since I mis understood the project I have implemented both the orjinal and the simplified upgma.

In [1]:
def pprint_dict(dct):
    """
    this function prints a dictionary like a matrix
    it is for debugging purposes, printing intermediate tables
    """
    for n in dct:
#         print(n, end='\t')
        for d in dct[n].values():
            print(d, end = '\t')
        print()

#### Task 1

#### read the distant matrix to variable dm

In [2]:

with open('distance_matrix.txt') as dmf:
    dm = [list(float(num) for num in line.rstrip()[1:-1].split(', ')) for line in dmf]
print(*dm, sep='\n')

[0.0]
[1.0, 0.0]
[13.0, 12.0, 0.0]
[17.0, 16.0, 10.0, 0.0]
[16.0, 15.0, 8.0, 1.0, 0.0]
[13.0, 12.0, 4.0, 5.0, 4.0, 0.0]
[12.0, 11.0, 6.0, 11.0, 10.0, 6.0, 0.0]


#### Complete distant matrix to a full symmetrical matrix, put it into full_dm variable

In [3]:
full_dm = []
max_ = len(max(dm, key=len))
for i in range(max_):
    row = []
    for j in range(max_):
        if i>=j:
            row.append(dm[i][j])
        else:
            row.append(dm[j][i])
    full_dm.append(row)
full_dm
        

[[0.0, 1.0, 13.0, 17.0, 16.0, 13.0, 12.0],
 [1.0, 0.0, 12.0, 16.0, 15.0, 12.0, 11.0],
 [13.0, 12.0, 0.0, 10.0, 8.0, 4.0, 6.0],
 [17.0, 16.0, 10.0, 0.0, 1.0, 5.0, 11.0],
 [16.0, 15.0, 8.0, 1.0, 0.0, 4.0, 10.0],
 [13.0, 12.0, 4.0, 5.0, 4.0, 0.0, 6.0],
 [12.0, 11.0, 6.0, 11.0, 10.0, 6.0, 0.0]]

Read the labels into labels variable,
also define a mapping for matching labels with indices

In [4]:
# read labels
with open('labels.txt') as lf:
    labels = [frozenset((s[1:-1], )) for s in lf.read().rstrip()[1:-1].split(', ')]
mapping = {label:i for (i,label) in enumerate(labels)}
rev_mapping = {i:label for (i,label) in enumerate(labels)}

In [5]:
print(labels)
print(mapping)
print(rev_mapping)

[frozenset({'Man'}), frozenset({'Monkey'}), frozenset({'Dog'}), frozenset({'Horse'}), frozenset({'Donkey'}), frozenset({'Pig'}), frozenset({'Rabbit'})]
{frozenset({'Man'}): 0, frozenset({'Monkey'}): 1, frozenset({'Dog'}): 2, frozenset({'Horse'}): 3, frozenset({'Donkey'}): 4, frozenset({'Pig'}): 5, frozenset({'Rabbit'}): 6}
{0: frozenset({'Man'}), 1: frozenset({'Monkey'}), 2: frozenset({'Dog'}), 3: frozenset({'Horse'}), 4: frozenset({'Donkey'}), 5: frozenset({'Pig'}), 6: frozenset({'Rabbit'})}


Prepare a history dictionary for keeping string representation of intermediate clusters

In [6]:
history = dict()
for i, c in rev_mapping.items():
#     print(i,c)
    history[c] = list(c)[0]
print(history)

{frozenset({'Man'}): 'Man', frozenset({'Monkey'}): 'Monkey', frozenset({'Dog'}): 'Dog', frozenset({'Horse'}): 'Horse', frozenset({'Donkey'}): 'Donkey', frozenset({'Pig'}): 'Pig', frozenset({'Rabbit'}): 'Rabbit'}


Hold distances in a dictionary so that it is easy to find and remove distances

In [7]:
dist_dict = {label_set:dict() for label_set in labels}
for i, label_set1 in enumerate(labels):
    for j, label_set2 in enumerate(labels):
        if i<j:
            continue
        try:
            dist_dict[label_set1][label_set2]=dm[i][j]
        except IndexError:
            dist_dict[label_set1][label_set2]=dm[j][i]

In [8]:
pprint_dict(dist_dict)

0.0	
1.0	0.0	
13.0	12.0	0.0	
17.0	16.0	10.0	0.0	
16.0	15.0	8.0	1.0	0.0	
13.0	12.0	4.0	5.0	4.0	0.0	
12.0	11.0	6.0	11.0	10.0	6.0	0.0	


#### Task 2

In [9]:
def argmin_on_dict(dct, ignore_zero=True):
    """
    Finds the smallest distance and return it as well as the pair of that distance 
    ignores zeros by default
    """
    min_ = float('inf')
    closests = (None, None)
    for i, c1 in enumerate(dct):
        for j, c2 in enumerate(dct[c1]):
            dist = dct[c1][c2]
            if ignore_zero and dist==0:
                continue
            if dist < min_:
                min_ = dist
                closests = (c1, c2)
    return min_, closests

In [10]:
argmin_on_dict(dist_dict, ignore_zero=True)

(1.0, (frozenset({'Monkey'}), frozenset({'Man'})))

#### Task 3

Below 2 cells are the orijinal upgma. you can find the simplified upgma later down.

In [11]:
orijinal_to_print = []
step = 1
print('--------------------{}----------------------'.format(0))
pprint_dict(dist_dict)
while True:
    min_, (c1, c2) = argmin_on_dict(dist_dict, ignore_zero=True)
    if min_ == float('inf'): # break if no distances are left, meaning all the clusters are merged already
        break
    new_cluster = c1.union(c2)
    history[new_cluster] = "({}, {})".format(history[c2],history[c1])
    orijinal_to_print.append(history[new_cluster])

    # calculate new distances
    new_dists = {}
    for c_other in dist_dict:
        if not (c_other == c1 or c_other == c2):
            total = 0
            count = len(c_other)*len(c1) + len(c_other)*len(c2)
            for sub_c_other in c_other:
                for sub_c1 in c1:
                    total += full_dm[mapping[frozenset((sub_c1,))]][mapping[frozenset((sub_c_other,))]]
                for sub_c2 in c2:
                    total += full_dm[mapping[frozenset((sub_c2,))]][mapping[frozenset((sub_c_other,))]]
            new_dists[c_other] = total/count
    new_dists[new_cluster] = 0

    # Add the new distnaces to the table
    dist_dict[new_cluster] = new_dists
    
    # remove old distances which are not needed anymore
    del dist_dict[c1]
    del dist_dict[c2]
    for c_other in dist_dict:
        try:
            del dist_dict[c_other][c1]
        except KeyError:
            pass
        try:
            del dist_dict[c_other][c2]
        except KeyError:
            pass
    print('--------------------{}----------------------'.format(step))
    step+=1
    pprint_dict(dist_dict)

--------------------0----------------------
0.0	
1.0	0.0	
13.0	12.0	0.0	
17.0	16.0	10.0	0.0	
16.0	15.0	8.0	1.0	0.0	
13.0	12.0	4.0	5.0	4.0	0.0	
12.0	11.0	6.0	11.0	10.0	6.0	0.0	
--------------------1----------------------
0.0	
10.0	0.0	
8.0	1.0	0.0	
4.0	5.0	4.0	0.0	
6.0	11.0	10.0	6.0	0.0	
12.5	16.5	15.5	12.5	11.5	0	
--------------------2----------------------
0.0	
4.0	0.0	
6.0	6.0	0.0	
12.5	12.5	11.5	0	
9.0	4.5	10.5	16.0	0	
--------------------3----------------------
0.0	
11.5	0	
10.5	16.0	0	
6.0	12.5	6.75	0	
--------------------4----------------------
0	
16.0	0	
12.166666666666666	8.0	0	
--------------------5----------------------
0	
13.7	0	
--------------------6----------------------
0	


Results of the original UPGMA are below, results of the simplfied UPGMA are at the bottom

In [12]:
for i, p in enumerate(orijinal_to_print):
    print("{}. {}".format(i+1, p))

1. (Man, Monkey)
2. (Horse, Donkey)
3. (Dog, Pig)
4. (Rabbit, (Dog, Pig))
5. ((Horse, Donkey), (Rabbit, (Dog, Pig)))
6. ((Man, Monkey), ((Horse, Donkey), (Rabbit, (Dog, Pig))))


#### Below is the simplified upgma algorithm.

In [13]:
del dist_dict

dist_dict = {label_set:dict() for label_set in labels}
for i, label_set1 in enumerate(labels):
    for j, label_set2 in enumerate(labels):
        if i<j:
            continue
        try:
            dist_dict[label_set1][label_set2]=dm[i][j]
        except IndexError:
            dist_dict[label_set1][label_set2]=dm[j][i]

simple_to_print = []
step = 1
print('--------------------{}----------------------'.format(0))
pprint_dict(dist_dict)
while True:
    min_, (c1, c2) = argmin_on_dict(dist_dict, ignore_zero=True)
    if min_ == float('inf'): # break if no distances are left, meaning all the clusters are merged already
        break
    new_cluster = c1.union(c2)
    history[new_cluster] = "({}, {})".format(history[c2],history[c1])
    simple_to_print.append(history[new_cluster])

    # calculate new distances
    new_dists = {}
    for c_other in dist_dict:
        if not (c_other == c1 or c_other == c2):
            try:
                d1 = dist_dict[c_other][c1]
            except:
                d1 = dist_dict[c1][c_other]
            try:
                d2 = dist_dict[c_other][c2]
            except:
                d2 = dist_dict[c2][c_other]               
            new_dists[c_other] = (d1+d2)/2
    new_dists[new_cluster] = 0

    # Add the new distnaces to the table
    dist_dict[new_cluster] = new_dists
    
    # remove old distances which are not needed anymore
    del dist_dict[c1]
    del dist_dict[c2]
    for c_other in dist_dict:
        try:
            del dist_dict[c_other][c1]
        except KeyError:
            pass
        try:
            del dist_dict[c_other][c2]
        except KeyError:
            pass
    print('--------------------{}----------------------'.format(step))
    step+=1
    pprint_dict(dist_dict)

--------------------0----------------------
0.0	
1.0	0.0	
13.0	12.0	0.0	
17.0	16.0	10.0	0.0	
16.0	15.0	8.0	1.0	0.0	
13.0	12.0	4.0	5.0	4.0	0.0	
12.0	11.0	6.0	11.0	10.0	6.0	0.0	
--------------------1----------------------
0.0	
10.0	0.0	
8.0	1.0	0.0	
4.0	5.0	4.0	0.0	
6.0	11.0	10.0	6.0	0.0	
12.5	16.5	15.5	12.5	11.5	0	
--------------------2----------------------
0.0	
4.0	0.0	
6.0	6.0	0.0	
12.5	12.5	11.5	0	
9.0	4.5	10.5	16.0	0	
--------------------3----------------------
0.0	
11.5	0	
10.5	16.0	0	
6.0	12.5	6.75	0	
--------------------4----------------------
0	
16.0	0	
12.0	8.625	0	
--------------------5----------------------
0	
14.0	0	
--------------------6----------------------
0	


#### Task 4 - just print intermediate clusters contained in a list
### THIS IS THE SIMPLIFIED UPGMA results as requested in the description

In [14]:
for i, p in enumerate(simple_to_print):
    print("{}. {}".format(i+1, p))

1. (Man, Monkey)
2. (Horse, Donkey)
3. (Dog, Pig)
4. (Rabbit, (Dog, Pig))
5. ((Horse, Donkey), (Rabbit, (Dog, Pig)))
6. ((Man, Monkey), ((Horse, Donkey), (Rabbit, (Dog, Pig))))


My results are the same as the given example in the description. But the sorting is different yet the clusters are the same.
